# To do today:

* ~Work on broken SMILES - figure out why they are broken and what to do~
* Read paper sent by Miles (https://www-nature-com.ezproxy.is.ed.ac.uk/articles/s41576-021-00434-9)
* Work on PyMol alignment script
* Essential Reading

# Notes for today:

* Wrote a script to count the number of files in a directory, seen below as "dir_file_counter.py".
* Used this script to count the output of PDB fiels from SMILES. There are 51571 decoys in "Decoy_Ligand_Smiles.csv", but 51,469 PDB files in the output directory, meaning 102 broken SMILES.
  * Also check the decoys pdbqt folder in the SCORCH sharepoint "Inputs for docking" folder. This file cotained 51570 decoy ligands in this file.
* Going to investigate which PDB codes present in the Decoy file are missing in the Active ligand file.
* Fixed issue with extra decoys, however 100 broken SMILES remain. Error code in box below.
* 2 more warning (not error) codes with smiles_to_pdb.py with rdkit "[23:11:56] UFFTYPER: Warning: hybridization set to SP3 for atom..." and [23:17:07] UFFTYPER: Unrecognized atom type: S_5+6 (26)"
* Found a "smiles cleaner" function written by Vincent here: https://github.com/LBLQMM/MACAW/blob/main/macaw/macaw.py
  * Deep cleaner did not work

## Broken Ligand error code

Traceback (most recent call last):
  File "smile_to_pdb.py", line 72, in <module>
    main()
  File "smile_to_pdb.py", line 59, in main
    AllChem.EmbedMolecule(mol)
Boost.Python.ArgumentError: Python argument types in
    rdkit.Chem.rdDistGeom.EmbedMolecule(NoneType)
did not match C++ signature:
    EmbedMolecule(RDKit::ROMol {lvalue} mol, RDKit::DGeomHelpers::EmbedParameters {lvalue} params)
    EmbedMolecule(RDKit::ROMol {lvalue} mol, unsigned int maxAttempts=0, int randomSeed=-1, bool clearConfs=True, bool useRandomCoords=False, double boxSizeMult=2.0, bool randNegEig=True, unsigned int numZeroFail=1, boost::python::dict {lvalue} coordMap={}, double forceTol=0.001, bool ignoreSmoothingFailures=False, bool enforceChirality=True, bool useExpTorsionAnglePrefs=True, bool useBasicKnowledge=True, bool printExpTorsionAngles=False, bool useSmallRingTorsions=False, bool useMacrocycleTorsions=False, unsigned int ETversion=1)

# Tasks completed today:

* Fixed the issue with the extra decoy SMILES in csv file with the script "missing_pdb_checker.py" as seen below. 
  * This script checks "Decoy_Ligand_Smiles.csv" for PDB codes that are not present in the "Active_Ligand_Smiles.csv". It saves the pdb code to a text file, then saves a new csv called "Updated_Decoy_Ligand_Smiles.csv". 
  * The "Updated_Decoy_Ligand_Smiles.csv" contains 51,020 decoys, which is in line with the figure given in the SCORCH paper.
* Used a script called "RDKit_Smile_check.py" to individually check some SMILEs (code below). Error message: "[22:57:35] Explicit valence for atom # 7 C, 6, is greater than permitted Could not create molecule from SMILES string: C[C@H](O)c1cn(N=C234CC[C@]25=C3C(NC(=O)C(C)(C)C)=CC4=C5)cc1NC(=O)N[C@H](C)C1=C(C(=O)O)OC=CC(Cl)=C1"
* Updated the smile_to_pdb.py script to add temporary hydrogens then remove before saving as this is what Miles did in his Lab book.
* Tried using smiles cleaner function (written below "smiles_cleaner_test.py").It failed to clean the SMILES tested from the broken SMILES list.

## missing_pdb_checker.py

In [ ]:
import pandas as pd

def main():
    # Load the data
    decoy_df = pd.read_csv('/home/s2451611/MScProject/SCORCH_SMILES/Decoy_Ligand_Smiles.csv')
    active_df = pd.read_csv('/home/s2451611/MScProject/SCORCH_SMILES/Active_Ligand_Smiles.csv')

    # Extract PDB codes from the decoy dataframe
    decoy_df['PDBCode'] = decoy_df['Ligand'].apply(lambda x: x.split('_')[0])

    # Find the PDB codes that are in the decoy dataframe but not in the active dataframe
    missing_pdb_codes = set(decoy_df['PDBCode']) - set(active_df['PDBCode'])

    # Write the missing PDB codes to a text file
    with open('Missing_PDBs.txt', 'w') as f:
        for pdb_code in missing_pdb_codes:
            f.write(f'{pdb_code}\n')
    
    # Remove rows with missing PDB codes from the decoy dataframe
    decoy_df = decoy_df[~decoy_df['PDBCode'].isin(missing_pdb_codes)]

    # Write the updated decoy dataframe to a new CSV file
    decoy_df.to_csv('/home/s2451611/MScProject/SCORCH_SMILES/Updated_Decoy_Ligand_Smiles.csv', index=False)
    return 


if __name__ == "__main__":
    main()

## RDKit_Smile_check.py

In [ ]:
from rdkit import Chem, rdBase, RDLogger
from rdkit.Chem import AllChem

smile = "C[C@H](O)c1cn(N=C234CC[C@]25=C3C(NC(=O)C(C)(C)C)=CC4=C5)cc1NC(=O)N[C@H](C)C1=C(C(=O)O)OC=CC(Cl)=C1"
mol = Chem.MolFromSmiles(smile)
if mol is None:
    print(f"Could not create molecule from SMILES string: {smile}")
else:
    AllChem.EmbedMolecule(mol)


## smiles_cleaner_test.py

In [ ]:
from rdkit import Chem
import re

def smiles_cleaner(smiles, return_idx=False, deep_clean=True):
    smiles = list(smiles)
    idx = []
    idx_bad = []
    clean_smiles = []
    for i, s in enumerate(smiles):
        try:
            if deep_clean:
                smi = s.replace(" ", "")
                smi = max(smi.split("."), key=len)
                for m in re.findall("\[.*?\]", smi):
                    if m not in ['[N+]', '[O-]']:
                        smi = smi.replace(m, m[1].upper())
                smi = smi.replace("@", "")
                smi = smi.replace("/C", "C")
                smi = smi.replace("\\C", "C")
                smi = smi.replace("/c", "c")
                smi = smi.replace("\\c", "c")
                
            else:
                smi = s
            
            m = Chem.MolFromSmiles(smi, sanitize=True)
        except:
            m = None

        if m is not None:
            idx.append(i)
            clean_smiles.append(smi)
        else:
            idx_bad.append(i)
            print(f"Warning: invalid SMILES in position {i}: {s}")

    if return_idx:
        return clean_smiles, idx, idx_bad
    else:
        return clean_smiles

# The SMILES string you want to clean
smiles_string = "NC(=O)CN123C(=O)[C@@H]4OCOCCNC5=[N@]16C21=[C@]63C(=CC=COC(=O)OCCN(CC(C(N)=O)=C4C(=O)O)C(=O)OO1)N5"

# Call the function with the SMILES string
cleaned_smiles = smiles_cleaner([smiles_string], deep_clean=True)

# Print the cleaned SMILES string
print(cleaned_smiles)


## To do tomorrow:
* Read paper sent by Miles (https://www-nature-com.ezproxy.is.ed.ac.uk/articles/s41576-021-00434-9)
* Work on PyMol alignment script
* Essential Reading
* Meeting with Miles - ask questions
* Find questions for Miles